# Tuning and deploy a foundation model


**Learning Objective**

1. Learn how to generate a JSONL file for PaLM tuning
1. Learn how to launch a tuning job on Vertex Pipeline
1. Learn how to deploy and query a tuned LLM
1. Learn how to evaluate a tuned LLM


Creating an LLM requires massive amounts of data, significant computing resources, and specialized skills. In this notebook, you'll learn how tuning allows you to customize a PaLM foundation model on Vertex Generative AI studio for more specific tasks or knowledge domains.
While the prompt design is excellent for quick experimentation, if training data is available, you can achieve higher quality by tuning the model. Tuning a model enables you to customize the model response based on examples of the task you want the model to perform.

For more details on tuning have a look at the [official documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

**Quota**: Tuning the `text-bison@001`  model uses the `tpu-v3-8` training resources and the accompanying quotas from your Google Cloud project. Each project has a default quota of eight v3-8 cores, which allows for one to two concurrent tuning jobs. If you want to run more concurrent jobs you need to request additional quota via the [Quotas page](https://console.cloud.google.com/iam-admin/quotas).

**Costs:** This tutorial uses a billable component of Google Cloud `Vertex AI Generative AI Studio`.
Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.


## Setup

In [1]:
import IPython

# The version of google-cloud-aiplatform needs to be >= 1.33.0
!pip install -q --upgrade --user \
    google-cloud-aiplatform \
    sequence-evaluate \
    sentence-transformers \
    rouge

# Restart the kernel
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import time

import pandas as pd
from google.cloud import aiplatform, bigquery
from seq_eval import SeqEval
from sklearn.model_selection import train_test_split
from vertexai.preview.language_models import TextGenerationModel

In [3]:
REGION = "us-central1"
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
BUCKET_NAME = PROJECT_ID
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
!gsutil ls $BUCKET_URI || gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## Training Data


In this notebook, we will be tuning the Vertex PaLM vertex using the Python SDK on a questions & answers dataset  from StackOverflow. 
Our first step will be to query the StackOverflow data on BigQuery Public Datasets, limiting to questions with the `python` tag, and `accepted` answers from 2020-01-01 only. 

We will limit the dataset to 1000 samples, 800 of which will be used to tune the LLM and the rest for evaluating the tuned model.
The second step will be to convert the dataset into a JSONL format, with one example per line, so that the tuning job can consume it.


The cell below contains  a helper function that lets you easily query BigQuery and return the results as a Pandas DataFrame:

In [4]:
def run_bq_query(sql):
    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    df = client_result.result().to_arrow().to_pandas()
    print(f"Finished job_id: {job_id}")

    return df

Next define the query.

In [5]:
query = """
SELECT CONCAT(q.title, q.body) as input_text, a.body AS output_text
FROM
    `bigquery-public-data.stackoverflow.posts_questions` q
JOIN
    `bigquery-public-data.stackoverflow.posts_answers` a
ON
    q.accepted_answer_id = a.id
WHERE
    q.accepted_answer_id IS NOT NULL AND
    REGEXP_CONTAINS(q.tags, "python") AND
    a.creation_date >= "2020-01-01"
LIMIT
    1000
"""

In [6]:
df = run_bq_query(query)
df.head()

Finished job_id: 14864704-b4b7-4f0b-ae64-a6b704033c3c


,input_text,output_text
0,append dataframe in nested loop<p>I have the f...,<p>I am not entirely sure if I understand your...
1,Python pandas find element of one column in li...,<p>You can do <code>apply</code>:</p>\n<pre><c...
2,How to add a minimum value constraint in Pyomo...,<p>figured it out. The two methods I described...
3,Producing Buffer Radius Polygons - Possible Pr...,<p>This is apparently an issue with <code>geov...
4,SMOTE for balancing data<p>I am trying to trai...,<p>You haven't given enough of your code or da...


There should be 1000 questions and answers.

In [7]:
print(len(df))

1000


Let's split the data into training and evaluation. To tune PaLM for a Q&A task we advise 100+ training examples. In this case you will use 800.

In [8]:
# split is set to 80/20
train, evaluation = train_test_split(df, test_size=0.2)
print(len(train))

800


For tuning, the training data first needs to be converted into a JSONL format, which is very easy in Pandas:

In [11]:
training_data_filename = "tune_data_stack_overflow_python_qa.jsonl"

train.to_json(training_data_filename, orient="records", lines=True)

Let's inspect the first line of the JSONL file we just created:

In [ ]:
!head -n 1 $training_data_filename

You can then export the local file to GCS, so that it can be used by Vertex AI for the tuning job.

In [ ]:
!gsutil cp $training_data_filename $BUCKET_URI

You can check to make sure that the file successfully transferred to your Google Cloud Storage bucket:

In [ ]:
TRAINING_DATA_URI = f"{BUCKET_URI}/{training_data_filename}"

!gsutil ls -al $TRAINING_DATA_URI

### Model Tuning
Now it's time to start to tune a model. You will use the Vertex AI SDK to submit our tuning job.

#### Recommended Tuning Configurations
✅ Here are some recommended configurations for tuning a foundation model based on the task, in this example Q&A. You can find more in the [documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-models).

Question Answering task:
- Make sure that your train dataset size is 100+
- Choose your training steps in the range 100-500. You can try more than one value to get the best performance on a particular dataset (e.g. 100, 200, 500)

In [68]:
aiplatform.init(project=PROJECT_ID, location=REGION)

model = TextGenerationModel.from_pretrained("text-bison@001")

Next it's time to start your tuning job. 

**Disclaimer:** tuning and deploying a model takes time.

In [ ]:
TRAIN_STEPS = 500
MODEL_NAME = f"asl-palm-text-tuned-model-{time.time()}"

model.tune_model(
    training_data=TRAINING_DATA_URI,
    model_display_name=MODEL_NAME,
    train_steps=TRAIN_STEPS,
    # Tuning can only happen in the "europe-west4" location for now
    tuning_job_location="europe-west4",
    # Model can only be deployed in the "us-central1" location for now
    tuned_model_location="us-central1",
)

print("Model name:", MODEL_NAME)

## Retrieve the tuned model from your Vertex AI Model registry


When your tuning job is finished, your model will be available on Vertex AI Model Registry. The next cell shows you how to list tuned models.

In [9]:
model = TextGenerationModel.from_pretrained("text-bison@001")
model.list_tuned_model_names()

['projects/115851500182/locations/us-central1/models/4267906115817177088',
 'projects/115851500182/locations/us-central1/models/7558911543518167040']

You can also use the Google Cloud Console UI to view all of your models in [Vertex AI Model Registry](https://console.cloud.google.com/vertex-ai/models?). 

It's time to get predictions. First you need to get the latest tuned model from the Vertex AI Model registry.

In [11]:
tuned_model = TextGenerationModel.get_tuned_model(
    model.list_tuned_model_names()[-1]
)

Now you can start sending a prompt to the API. Feel free to update the following prompt:

In [12]:
PROMPT = """
How can I store my TensorFlow checkpoint on Google Cloud Storage?

Python example:

"""

print(tuned_model.predict(PROMPT))

```python
import tensorflow as tf

# Create a GCS bucket
bucket = tf.gfile.GFile('gs://my-bucket/', 'w')

# Create a checkpoint directory
checkpoint_dir = 'gs://my-bucket/checkpoints/'

# Create a checkpoint file
checkpoint_file = os.path.join(checkpoint_dir, 'checkpoint')

# Create a saver
saver = tf.train.Saver()

# Save the checkpoint
saver.save(sess, checkpoint_file)

# Restore the


## Evaluation


It's essential to evaluate your model to understand its performance. Evaluation can be done in an automated way using evaluation metrics like F1, Bleu, or Rouge. You can also leverage human evaluation methods. Human evaluation methods involve asking humans to rate the quality of the LLM's answers. This can be done through crowdsourcing or by having experts evaluate the responses. Some standard human evaluation metrics include fluency, coherence, relevance, and informativeness. Often you want to choose a mix of evaluation metrics to get a good understanding of your model performance. 


Among other metrics we will compute the following two metrics that provide crude measures albeit automated of how two texts may have the same meaning: 
- [Blue](https://en.wikipedia.org/wiki/BLEU): The BLEU evaluation metric is a measure of the similarity between a machine-generated text and a human-written reference text.
- [Rouge](https://en.wikipedia.org/wiki/ROUGE_(metric)): The ROUGE evaluation metric is a measure of the overlap between a machine-generated text and a human-written reference text.


We will use  [sequence-evaluate](https://pypi.org/project/sequence-evaluate/) to to compute the scores.
Earlier in the notebook, you created a train and eval dataset. Now it's time to take some of the eval data. You will use the questions to get a response from our tuned model, and the answers we will use as a reference:
- **Candidates**: Answers generated by the tuned model.
- **References**: Original answers that we will use to compare


Let us first select a sample of our evaluation set:

In [13]:
# you can change the number of rows you want to use
EVAL_ROWS = 60

evaluation = evaluation.head(EVAL_ROWS)
evaluation.head()

,input_text,output_text
787,Changing some values in a row of pd.DataFrame ...,<p>You may want to try this usage of <code>loc...
254,Split Large csv File into multiple files depen...,<p>Here is one approach.</p>\n<pre><code>fn = ...
458,When to use the bitwise and operator (&)?<p>I ...,"<p>As the comments mentioned, <code>num&amp;1<..."
142,Pandas can't select index range as string date...,<p>The index dates were strings instead of dat...
42,Scale with Kivy ScatterLayout Doesn't Behave a...,<p>You have to scale the size of the <code>Sca...


The function in the cell below will query our tuned model using the `evaluation.input_text` and store the ground truth in `evaluation.output_text` in a DataFrame next to the model answers:

In [53]:
def create_eval_data(model, evaluation):
    model_answers = []

    for prompt in evaluation.input_text:
        response = model.predict(prompt)
        model_answers.append(response.text)

    eval_df = pd.DataFrame(
        {"candidate": model_answers, "reference": evaluation.output_text}
    )
    mask = eval_df.candidate == ""
    return eval_df[~mask]

In [54]:
eval_df = create_eval_data(model, evaluation)

In [55]:
eval_df.head()

,candidate,reference
787,The error is because you are trying to change ...,<p>You may want to try this usage of <code>loc...
254,```python\nimport csv\n\ndef split_csv(input_f...,<p>Here is one approach.</p>\n<pre><code>fn = ...
458,The bitwise and operator (&) is used to perfor...,"<p>As the comments mentioned, <code>num&amp;1<..."
142,The problem is that the index is a `DatetimeIn...,<p>The index dates were strings instead of dat...
42,The problem is that you are not setting the <c...,<p>You have to scale the size of the <code>Sca...


The function in the next cell computes a number of metrics (Rouge, Blue, etc.) useful to indicate whether two texts have the same meaning. It averages these scores over all the reference answers and those generated by our tuned model, giving scores that can serve as performance metrics for our model.

In [56]:
def compute_scores(eval_data):
    evaluator = SeqEval()
    reference = eval_data.reference.tolist()
    candidate = eval_data.candidate.tolist()
    return evaluator.evaluate(reference, candidate, verbose=False)

In [57]:
compute_scores(eval_df)

{'bleu_1': 0.11048626784959818,
 'bleu_2': 0.049981933337872736,
 'bleu_3': 0.029993136777644553,
 'bleu_4': 0.019459490228121507,
 'rouge_1_precision': 0.14123178063031,
 'rouge_1_recall': 0.24180495796915416,
 'rouge_1_f1': 0.15766779691203167,
 'rouge_2_precision': 0.03205610888282679,
 'rouge_2_recall': 0.057176760321351904,
 'rouge_2_f1': 0.03629933742380242,
 'rouge_l_precision': 0.12927872916655078,
 'rouge_l_recall': 0.22340454177548244,
 'rouge_l_f1': 0.14484886963077176,
 'inter_dist1': 0.0015513048048969688,
 'inter_dist2': 0.03906219163510438,
 'intra_dist1': 0.07823671070794777,
 'intra_dist2': 0.34727942840194825,
 'semantic_textual_similarity': 0.5892551027495285}

Given two versions of the model (possibly tuned with a different amount of data or training steps), you can now compare the scores to decide which one is the best. However, remember that these automated metrics are very crude proxy of human assessment. 

## Acknowledgement 

This notebook is adapted from a [tutorial](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/tuning/getting_started_tuning.ipynb)
written by Polong Lin.

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

     https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.